In [122]:
import pandas as pd
from sklearn.preprocessing import OrdinalEncoder
from sklearn import tree
from datetime import date
from datetime import datetime
from fastapi import Request, FastAPI
from sklearn.model_selection import train_test_split
from pydantic import BaseModel
import json
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeClassifier
from fastapi import FastAPI
from pydantic import BaseModel

In [123]:
df = pd.read_excel("2. Datos para entrenar.xlsx",engine='openpyxl')
filtro=[]
for i,j in enumerate(df['fecha_nacimiento']):
    filter=type(df['fecha_nacimiento'][0])
    if type(j)==filter:
        filtro.append(True)
    else:
        filtro.append(False) 
df=df[filtro]
edad=[]
for k,h in enumerate(df['fecha_nacimiento']):
    edad.append(2022-h.year)
df['Edad']=edad

In [124]:
salidas1= df[['consulta_id','cups_solictud','descripcion','cantidad','prescripcion','tipo','codigo_medicamento','descripcion_medicamento','cantidad_medicamento','prescripcion_medicamento']]
entradas= df.drop(['diagnostico_secundario','diagnostico_terciario','cuarto_diagnostico','presentacion','fecha_nacimiento','cita_id','id_solicitud_enviada','medicamento_enviado',"TEST_HERRERA_Y_HURTADO",'cups_solictud','descripcion','cantidad','prescripcion','tipo','codigo_medicamento','descripcion_medicamento','cantidad_medicamento','prescripcion_medicamento'], axis=1)
salidas1=salidas1.fillna(0)
entradas=entradas.fillna(0)
cups_solictud=[]
descripcion=[]
cantidad=[]
prescripcion=[]
tipo=[]
codigo_medicamento=[]
descripcion_medicamento=[]
cantidad_medicamento=[]
prescripcion_medicamento=[]
#presentacion=[]
enes=[]

In [125]:
for n in salidas1['consulta_id'].drop_duplicates():
    if len(salidas1[salidas1['consulta_id']==n])>0:
        cups_solictud1=' '
        descripcion1=' '
        cantidad1=' '
        prescripcion1=' '
        tipo1=' '
        codigo_medicamento1=' '
        descripcion_medicamento1=''
        cantidad_medicamento1=''
        prescripcion_medicamento1=''
        #presentacion1=''
        for k in range(len(salidas1[salidas1['consulta_id']==n])):
            cups_solictud1=cups_solictud1+'-'+str(salidas1[salidas1['consulta_id']==n]['cups_solictud'].to_list()[k])
            descripcion1=descripcion1+'-'+str(salidas1[salidas1['consulta_id']==n]['descripcion'].to_list()[k])
            cantidad1=cantidad1+'-'+str(salidas1[salidas1['consulta_id']==n]['cantidad'].to_list()[k])
            prescripcion1=prescripcion1+'-'+str(salidas1[salidas1['consulta_id']==n]['prescripcion'].to_list()[k])
            tipo1=tipo1+'-'+str(salidas1[salidas1['consulta_id']==n]['tipo'].to_list()[k])
            codigo_medicamento1=codigo_medicamento1+'-'+str(salidas1[salidas1['consulta_id']==n]['codigo_medicamento'].to_list()[k])
            descripcion_medicamento1=descripcion_medicamento1+'-'+str(salidas1[salidas1['consulta_id']==n]['descripcion_medicamento'].to_list()[k])
            cantidad_medicamento1=cantidad_medicamento1+'-'+str(salidas1[salidas1['consulta_id']==n]['cantidad_medicamento'].to_list()[k])
            prescripcion_medicamento1=prescripcion_medicamento1+'-'+str(salidas1[salidas1['consulta_id']==n]['prescripcion_medicamento'].to_list()[k])
            #presentacion1=presentacion1+'-'+str(salidas1[salidas1['consulta_id']==n]['presentacion'].to_list()[k])
        cups_solictud.append(cups_solictud1)
        descripcion.append(descripcion1)
        cantidad.append(cantidad1)
        prescripcion.append(prescripcion1)
        tipo.append(tipo1)
        codigo_medicamento.append(codigo_medicamento1)
        descripcion_medicamento.append(descripcion_medicamento1)
        cantidad_medicamento.append(cantidad_medicamento1)
        prescripcion_medicamento.append(prescripcion_medicamento1)
        #presentacion.append(presentacion1)
        enes.append(n)
    else:
        continue
salidas = pd.DataFrame()
salidas['consulta_id']=enes
salidas['cups_solictud']=cups_solictud
salidas['descripcion']=descripcion
salidas['cantidad']=cantidad
salidas['prescripcion']=prescripcion
salidas['tipo']=tipo
salidas['codigo_medicamento']=codigo_medicamento
salidas['descripcion_medicamento']=descripcion_medicamento
salidas['cantidad_medicamento']=cantidad_medicamento
salidas['prescripcion_medicamento']=prescripcion_medicamento
#salidas['presentacion']=presentacion

In [126]:
for j in entradas['consulta_id'].drop_duplicates():
    if j in enes:
        continue
    else:
        entradas = entradas.drop(entradas[entradas[j]==True].index)
entradas['consulta_id']=entradas['consulta_id'].drop_duplicates().dropna()
entradas=entradas.dropna()
entradas=entradas.sort_values(by=['consulta_id'])
salidas=salidas.sort_values(by=['consulta_id'])
entradas=entradas.drop(columns=['consulta_id'])
salidas=salidas.drop(columns=['consulta_id'])
entradas['T_A'] = entradas['T_A'].str.replace('-','/')
t_a = entradas["T_A"].str.split('/', expand=True)
t_a.columns = ['T_A1', 'T_A2']
entradas = pd.concat([entradas, t_a], axis=1)
entradas['TEST_DE_FINDRISK']=entradas['TEST_DE_FINDRISK'].astype(str).apply(lambda x: x[:2]).str.replace(':',' ').astype(float)
entradas['TASA_DE_FILTRACION_GLOMERULAR']=entradas['TASA_DE_FILTRACION_GLOMERULAR'].astype(str).apply(lambda x: x[:2]).astype(float)
entradas['TEST_DE_FRAMINGHAM']=entradas['TEST_DE_FRAMINGHAM'].astype(str).apply(lambda x: x[:1]).str.replace('%',' ').astype(float)
salidas=salidas.fillna(0)
entradas=entradas.fillna(0)
entradas=entradas.drop(['T_A'], axis=1)

In [127]:
clf1=tree.DecisionTreeClassifier(criterion='entropy',max_depth=45,ccp_alpha=0.0005,min_samples_leaf=35)
clf2=tree.DecisionTreeClassifier(criterion='entropy',max_depth=45,ccp_alpha=0.0005,min_samples_leaf=35)

In [128]:
#import pickle

#pkl_filename = "pickle_model.pkl"
#with open(pkl_filename, 'wb') as file:
#    pickle.dump(clf1, file)
#pkl_filename = "pickle_model2.pkl"
#with open(pkl_filename, 'wb') as file:
#    pickle.dump(clf2, file)

In [129]:
tablas_enc = dict()
i=0
for j in entradas.columns.tolist():
    entradas[j]=entradas[j].fillna(0)
    if entradas[j].dtype=='O':
        entradas[j]=entradas[j].astype(str)
        tablas_enc[j] = OrdinalEncoder(categories=[entradas[j].unique().tolist()])
        tablas_enc[j].fit(entradas[[j]])
        entradas[j+'cat']= tablas_enc[j].transform(entradas[[j]])
    else:
        entradas[j+'cat'] = entradas[j]
        continue
for i in salidas.columns.tolist():
    salidas[i]=salidas[i].fillna(0)
    if salidas[i].dtype=='O':
        salidas[i]=salidas[i].astype(str)
        tablas_enc[i]=OrdinalEncoder(categories=[salidas[i].unique().tolist()])
        tablas_enc[i].fit(salidas[[i]])
        salidas[i+'cat'] = tablas_enc[i].transform(salidas[[i]])
    else:
        salidas[i+'cat'] = salidas[i]
        continue
salidas_cat=[]
for k in salidas.columns.tolist():
    if 'cat' in k:
        salidas_cat.append(k)
    else:
        continue
entradas_cat=[]
for h in entradas.columns.tolist():
    if 'cat' in h:
        entradas_cat.append(h)
    else:
        continue

In [130]:
salidas_cat1=[]
salidas_cat1.append('cups_solictudcat')
salidas_cat1.append('descripcioncat')
salidas_cat1.append('cantidadcat')
salidas_cat1.append('prescripcioncat')
salidas_cat1.append('tipocat')
salidas_cat2=[]
salidas_cat2.append('codigo_medicamentocat')
salidas_cat2.append('descripcion_medicamentocat')
salidas_cat2.append('cantidad_medicamentocat',)
salidas_cat2.append('prescripcion_medicamentocat')

In [131]:
Y1=salidas[salidas_cat1]
X1=entradas[entradas_cat]
Y2=salidas[salidas_cat2]
X2=salidas[salidas_cat1]
X_train1, X_test1, y_train1, y_test1 = train_test_split(X1, Y1, test_size=0.20)
X_train2, X_test2, y_train2, y_test2 = train_test_split(X2, Y2, test_size=0.20)

In [132]:
clf1=clf1.fit(X_train1, y_train1)
clf2=clf2.fit(X_train2, y_train2)

In [133]:
dato_en=[]
dato_en.append(int('30'))
dato_en.append(tablas_enc['sexo'].transform([['M']])[0][0])
dato_en.append(float('80'))
dato_en.append(tablas_enc['grupo_sanguineo'].fit_transform([['O-']])[0][0])
dato_en.append(float('100'))
dato_en.append(float('140'))
dato_en.append(int('20'))#F_C
dato_en.append(int('0'))#F_R
dato_en.append(float('1'))#Test de frindrisk
dato_en.append(float('123'))#Tasa de filtracion
dato_en.append(float('0'))#Test de framingan ,porcentaje
dato_en.append(tablas_enc['diagnostico_principal'].fit_transform([['N390']]))#diagnostico_principal
#dato_en.append(tablas_enc['diagnostico_secundario'].fit_transform([[item.diagnostico_secundariocat]]))#diagnostico_secundario
#dato_en.append(tablas_enc['diagnostico_terciario'].fit_transform([[item.diagnostico_terciariocat]]))#diagnostico_terciario
#dato_en.append(tablas_enc['cuarto_diagnostico'].fit_transform([[item.cuarto_diagnosticocat]]))#cuarto_diagnostico
dato_en.append(int('120'))#T_a1
dato_en.append(int('50'))#T_a2
prediccion= clf1.predict([dato_en])
lists = prediccion.tolist()
prediccion2= clf2.predict(lists)
lists2 = prediccion2.tolist()
lists = prediccion.tolist()
respuesta4=0
respuesta4={
            "cups_solicitud":tablas_enc['cups_solictud'].inverse_transform([[prediccion[0][0]]])[0][0],
            "cups_solicitud_probabilidad":max(clf1.predict_proba([dato_en])[0][0]),
            "descripcion" :tablas_enc['descripcion'].inverse_transform([[prediccion[0][1]]])[0][0],
            "descripcion_probabilidad":max(clf1.predict_proba([dato_en])[1][0]),
            "cantidad":tablas_enc['cantidad'].inverse_transform([[prediccion[0][2]]])[0][0],
            "cantidad_probabilidad":max(clf1.predict_proba([dato_en])[2][0]),
            "prescripcion": tablas_enc['prescripcion'].inverse_transform([[prediccion[0][3]]])[0][0][0][0],
            "prescripcion_probabilidad":max(clf1.predict_proba([dato_en])[3][0]),
            "tipo": tablas_enc['tipo'].inverse_transform([[prediccion[0][4]]])[0][0],
            "tipo_probabilidad":max(clf1.predict_proba([dato_en])[4][0]),
            "codigo_medicamento":tablas_enc['codigo_medicamento'].inverse_transform([[prediccion2[0][0]]])[0][0],
            "codigo_medicamento_probabilidad":max(clf2.predict_proba([dato_en])[0][0]),
            "descripcion_medicamento":tablas_enc['descripcion_medicamento'].inverse_transform([[prediccion2[0][1]]])[0][0],
            "descripcion_medicamento_probabilidad":max(clf2.predict_proba([dato_en])[1][0]),
            "cantidad_medicamento":tablas_enc['cantidad_medicamento'].inverse_transform([[prediccion2[0][2]]])[0][0],
            "cantidad_medicamento_probabilidad":max(clf2.predict_proba([dato_en])[2][0]),
            "prescripcion_medicamento":tablas_enc['prescripcion_medicamento'].inverse_transform([[prediccion2[0][3]]])[0][0],
            "prescripcion_medicamento_probabilidad":max(clf2.predict_proba([dato_en])[3][0]),
            #"presentacion":tablas_enc['presentacion'].inverse_transform([[prediccion2[0][4]]])[0][0]
        }

c:\Users\vicma\miniconda3\envs\new\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but OrdinalEncoder was fitted with feature names
  warnings.warn(
c:\Users\vicma\miniconda3\envs\new\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but DecisionTreeClassifier was fitted with feature names
  warnings.warn(
c:\Users\vicma\miniconda3\envs\new\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but DecisionTreeClassifier was fitted with feature names
  warnings.warn(


In [136]:
prueba=clf1.predict(X_test1)
prueba2=clf2.predict(X_test2)

In [137]:
prueba=prueba.tolist()
cups_solictud_prueba=[]
descripcion_prueba=[]
cantidad_prueba=[]
prescripcion_prueba=[]
tipo_prueba=[]
codigo_medicamento_prueba=[]
descripcion_medicamento_prueba=[]
cantidad_medicamento_prueba=[]
prescripcion_medicamento_prueba=[]
#presentacion_prueba=[]
for i in range(0,len(prueba)):
    cups_solictud_prueba.append(prueba[i][0])
    descripcion_prueba.append(prueba[i][1])
    cantidad_prueba.append(prueba[i][2])
    prescripcion_prueba.append(prueba[i][3])
    tipo_prueba.append(prueba[i][4])
    codigo_medicamento_prueba.append(prueba2[i][0])
    descripcion_medicamento_prueba.append(prueba2[i][1])
    cantidad_medicamento_prueba.append(prueba2[i][2])
    prescripcion_medicamento_prueba.append(prueba2[i][3])
    #presentacion_prueba.append(prueba[i][9])

In [138]:
(y_test1['prescripcioncat']==prescripcion_prueba).value_counts()

False    52
True     36
Name: prescripcioncat, dtype: int64

In [139]:
prueba

[[25.0, 25.0, 9.0, 9.0, 10.0],
 [64.0, 64.0, 9.0, 9.0, 10.0],
 [66.0, 66.0, 9.0, 9.0, 10.0],
 [31.0, 31.0, 9.0, 9.0, 10.0],
 [82.0, 82.0, 9.0, 9.0, 10.0],
 [64.0, 64.0, 9.0, 9.0, 10.0],
 [31.0, 31.0, 9.0, 9.0, 10.0],
 [66.0, 66.0, 9.0, 9.0, 10.0],
 [64.0, 64.0, 9.0, 9.0, 10.0],
 [64.0, 64.0, 9.0, 9.0, 10.0],
 [66.0, 66.0, 9.0, 9.0, 10.0],
 [64.0, 64.0, 9.0, 9.0, 10.0],
 [31.0, 31.0, 9.0, 9.0, 10.0],
 [31.0, 31.0, 9.0, 9.0, 10.0],
 [64.0, 64.0, 9.0, 9.0, 10.0],
 [13.0, 13.0, 9.0, 9.0, 10.0],
 [82.0, 82.0, 9.0, 9.0, 10.0],
 [66.0, 66.0, 9.0, 9.0, 10.0],
 [13.0, 13.0, 9.0, 9.0, 10.0],
 [66.0, 66.0, 9.0, 9.0, 10.0],
 [64.0, 64.0, 9.0, 9.0, 10.0],
 [66.0, 66.0, 9.0, 9.0, 10.0],
 [82.0, 82.0, 9.0, 9.0, 10.0],
 [31.0, 31.0, 9.0, 9.0, 10.0],
 [64.0, 64.0, 9.0, 9.0, 10.0],
 [31.0, 31.0, 9.0, 9.0, 10.0],
 [66.0, 66.0, 9.0, 9.0, 10.0],
 [64.0, 64.0, 9.0, 9.0, 10.0],
 [64.0, 64.0, 9.0, 9.0, 10.0],
 [66.0, 66.0, 9.0, 9.0, 10.0],
 [82.0, 82.0, 9.0, 9.0, 10.0],
 [31.0, 31.0, 9.0, 9.0, 10.0],
 [64.0, 

In [140]:
prueba2

array([[ 15.,  15.,  15.,  14.],
       [ 77.,  77.,  73.,  74.],
       [ 77.,  77.,  73.,  74.],
       [ 15.,  15.,  15.,  14.],
       [ 63.,  63.,  59.,  59.],
       [ 15.,  15.,  15.,  14.],
       [ 95., 100.,  68.,  70.],
       [ 15.,  15.,  15.,  14.],
       [ 95., 100.,  68.,  70.],
       [ 95., 100.,  68.,  70.],
       [ 45.,  45.,  41.,  41.],
       [ 95., 100.,  68.,  70.],
       [ 63.,  63.,  59.,  59.],
       [ 95., 100.,  68.,  70.],
       [ 95., 100.,  68.,  70.],
       [ 15.,  15.,  15.,  14.],
       [ 63.,  63.,  59.,  59.],
       [ 15.,  15.,  15.,  14.],
       [ 29.,  29.,  25.,  25.],
       [ 29.,  29.,  25.,  25.],
       [ 15.,  15.,  15.,  14.],
       [ 15.,  15.,  15.,  14.],
       [ 15.,  15.,  15.,  14.],
       [ 15.,  15.,  15.,  14.],
       [ 63.,  63.,  59.,  59.],
       [ 15.,  15.,  15.,  14.],
       [ 63.,  63.,  59.,  59.],
       [ 63.,  63.,  59.,  59.],
       [ 15.,  15.,  15.,  14.],
       [ 77.,  77.,  73.,  74.],
       [ 9

In [141]:
from joblib import dump
dump(clf1, 'clf1.joblib')
dump(clf2, 'clf2.joblib')
dump(tablas_enc, 'tables.joblib')

['tables.joblib']